In [1]:
from azure.identity import DefaultAzureCredential, get_bearer_token_provider
from openai import AzureOpenAI
import os
from dotenv import load_dotenv


# Load environment variables from .env file
load_dotenv()


True

In [4]:

token_provider = get_bearer_token_provider(
    DefaultAzureCredential(), "https://cognitiveservices.azure.com/.default"
)

api_version = os.environ["AZURE_OPENAI_API_VERSION"]
endpoint = os.environ["AZURE_OPENAI_ENDPOINT"]

client = AzureOpenAI(
    api_version=api_version,
    azure_ad_token_provider=token_provider,
)
response = client.responses.create(
    model=os.environ["AZURE_OPENAI_MODEL_NAME"],
    input="未来のテクノロジートレンドを3つ挙げてください"
)
print(response.output_text)

未来のテクノロジートレンドとして、以下の3つが注目されています：

1. **人工知能と機械学習（AI/ML）**：
   - **自動運転車**：交通システムの効率化や安全性の向上を目指して、自動運転技術は急速に進化しています。
   - **医療分野での応用**：診断支援や治療計画の立案、さらには新薬の開発においてAIの活用が進むでしょう。
   - **自然言語処理**：AIを用いた翻訳、コンテンツ生成、カスタマーサポートなどの分野で大きな進展が見込まれます。

2. **ブロックチェーンと分散型技術**：
   - **金融サービス（Decentralized Finance, DeFi）**：従来の金融システムに変わる、新たな金融インフラとして期待されています。
   - **サプライチェーン管理**：透明性と追跡可能性の向上に寄与し、偽造品の防止や効率的な物流管理を可能にします。
   - **デジタルアイデンティティ**：個人情報を安全に管理し、インターネット上でのアイデンティティ確認を簡単に行う手段として普及が進むでしょう。

3. **量子コンピューティング**：
   - **高性能計算**：複雑な問題を従来のコンピュータでは不可能な速度で解決するため、物理学、化学、バイオインフォマティクスなどの分野で革新的な成果が見込まれます。
   - **暗号技術**：現在の暗号化技術を破る能力を持つため、新たなセキュリティプロトコルの開発が加速するでしょう。
   - **機械学習の高速化**：より高次元のデータを処理することで、AI/MLアルゴリズムの効率と精度が大幅に向上する可能性があります。

これらの技術は、私たちの生活やビジネスの在り方を根本的に変えていく潜在力を持っています。


In [ ]:
inputs = [{"type": "message", "role": "user", "content": "Define and explain the concept of catastrophic forgetting?"}] 
  
response = client.responses.create(  
    model=os.environ["AZURE_OPENAI_MODEL_NAME"], # replace with your model deployment name  
    input=inputs  
)  
  
inputs += response.output

inputs.append({"role": "user", "type": "message", "content": "Explain this at a level that could be understood by a college freshman"}) 
               

second_response = client.responses.create(  
    model="gpt-4o",  
    input=inputs
)  
      
print(second_response.model_dump_json(indent=2)) 

In [ ]:
from openai import OpenAI
client = OpenAI(base_url=os.environ["AZURE_OPENAI_ENDPOINT"])

response = client.responses.create(
    model="gpt-4o",
    input=[
        {"role": "user", "content": "what teams are playing in this image?"},
        {
            "role": "user",
            "content": [
                {
                    "type": "input_image",
                    "image_url": "https://upload.wikimedia.org/wikipedia/commons/3/3b/LeBron_James_Layup_%28Cleveland_vs_Brooklyn_2018%29.jpg"
                }
            ]
        }
    ]
)

print(response.output_text)